In [22]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #<- for the common server(SSH)

In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import numpy as np

In [24]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [26]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
#override fc layer to 10 labels
model.fc = nn.Linear(2048, 10)
model = model.to(device)

loss_fun = nn.CrossEntropyLoss()
lr = 1e-3
optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
epochs = 5

def train(model, epochs, loss_fun, optimizer, trainloader):
    loss = []
    acc = []

    for _ in range(epochs):
        epoch_loss = 0
        epoch_acc = []
        model.train()
        for batch in tqdm(trainloader):
            input, labels = batch
            input = input.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            model_output = model(input)
            result = torch.argmax(model_output, dim=1)
            step_loss_val = loss_fun(model_output, labels)
            epoch_loss += step_loss_val.item()
            batch_acc = torch.count_nonzero(result == labels).item()
            batch_acc /= len(labels)
            epoch_acc.append(batch_acc)
            step_loss_val.backward()
            optimizer.step()
        epoch_loss /= len(trainloader)
        epoch_acc = np.mean(epoch_acc)
        loss.append(epoch_loss)
        acc.append(epoch_acc)

    return loss, acc

def test(model, loss_fun, testloader):
    loss = []
    acc = []
    epoch_loss = 0
    epoch_acc = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(testloader):
            input, labels = batch
            input = input.to(device)
            labels = labels.to(device)
            model_output = model(input)
            result = torch.argmax(model_output, dim=1)
            step_loss_val = loss_fun(model_output, labels)
            epoch_loss += step_loss_val.item()
            batch_acc = torch.count_nonzero(result == labels).item()
            batch_acc /= len(labels)
            epoch_acc.append(batch_acc)
        epoch_loss /= len(testloader)
        epoch_acc = np.mean(epoch_acc)
        loss.append(epoch_loss)
        acc.append(epoch_acc)

    return loss, acc

loss, acc = train(model, epochs, loss_fun, optimizer, trainloader)
print(loss)
print(acc)